In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv


# 망한 노트 회고....


- 기본적인 머신러닝의 매커니즘은 파악은 됐지만 아직 메소들을 사용하는데 미숙해 하나씩 사용해봐야할 듯
- 데이터 형식에 따라 변형시켜서 파라미터로 넣어야하는 경우가 있었다 xgboost 사용할 떄
- 뒤늦게 알게된 사실은 pipeline을 사용할 때는 train data와 column이 같아야하는 것 같다. class를 같이 전처리 할려고하니 에러... 이제 target은 따로 전처리하자
- target 같은 경우는 미리 labelencoder로 해놓는게 좋은것 같다. 나중에 나지 원복을 해야하기 떄문에 이번 competition에서는 
- labelencoder를 두번쓰면...어떻게 되는지 궁금하다. 호출된 labelencoder를 사용하면 문제는 없을 거 같음
- 오늘은 여기까지 내일은 class 전처리부터 해보기로 하자

In [30]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd 
import missingno as msno

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import matthews_corrcoef, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer


from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

random_state = 42

%matplotlib inline

# 데이터 가벼오고 기본적인 파악

In [3]:
train_data = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s4e8/test.csv")
submission =pd.read_csv("/kaggle/input/playground-series-s4e8/sample_submission.csv")

In [4]:
print(train_data.shape)
train_data.head()

(3116945, 22)


,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [5]:
print(test_data.shape)
test_data.tail()

(2077964, 21)


,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
2077959,5194904,0.88,x,g,w,f,a,d,w,2.67,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,u
2077960,5194905,3.12,x,s,w,f,d,c,w,2.69,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
2077961,5194906,5.73,x,e,e,f,a,NaN,w,6.16,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
2077962,5194907,5.03,b,g,n,f,a,d,g,6.00,...,NaN,s,g,NaN,NaN,f,f,NaN,d,a
2077963,5194908,15.51,f,NaN,w,f,d,c,y,2.69,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,w


In [6]:
# 데이터 타입 파악
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116945 entries, 0 to 3116944
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   class                 object 
 2   cap-diameter          float64
 3   cap-shape             object 
 4   cap-surface           object 
 5   cap-color             object 
 6   does-bruise-or-bleed  object 
 7   gill-attachment       object 
 8   gill-spacing          object 
 9   gill-color            object 
 10  stem-height           float64
 11  stem-width            float64
 12  stem-root             object 
 13  stem-surface          object 
 14  stem-color            object 
 15  veil-type             object 
 16  veil-color            object 
 17  has-ring              object 
 18  ring-type             object 
 19  spore-print-color     object 
 20  habitat               object 
 21  season                object 
dtypes: float64(3), int64(1), object(18)
memory

In [7]:
# NaN 값을 갖는 columns
((train_data.isnull().sum() /train_data.shape[0]) *100).sort_values()

id                       0.000000
stem-width               0.000000
stem-height              0.000000
season                   0.000000
class                    0.000000
cap-diameter             0.000128
does-bruise-or-bleed     0.000257
cap-color                0.000385
has-ring                 0.000770
stem-color               0.001219
cap-shape                0.001283
habitat                  0.001444
gill-color               0.001829
ring-type                4.134818
gill-attachment         16.809280
cap-surface             21.528227
gill-spacing            40.373988
stem-surface            63.551362
veil-color              87.936970
stem-root               88.452732
spore-print-color       91.425482
veil-type               94.884350
dtype: float64

In [8]:
# NaN 값을 갖는 columns
((test_data.isnull().sum() /test_data.shape[0]) *100).sort_values()

id                       0.000000
stem-width               0.000000
season                   0.000000
stem-height              0.000048
cap-diameter             0.000337
does-bruise-or-bleed     0.000481
cap-color                0.000626
has-ring                 0.000914
stem-color               0.001011
habitat                  0.001203
cap-shape                0.001492
gill-color               0.002358
ring-type                4.148051
gill-attachment         16.834796
cap-surface             21.506821
gill-spacing            40.404694
stem-surface            63.595327
veil-color              87.880445
stem-root               88.452543
spore-print-color       91.417224
veil-type               94.878689
dtype: float64

In [9]:
categorical_col = train_data.select_dtypes(include=['object']).columns

print("unique value")

for col in categorical_col:
    print(f"{col} : {train_data[col].nunique()} ")

unique value
class : 2 
cap-shape : 74 
cap-surface : 83 
cap-color : 78 
does-bruise-or-bleed : 26 
gill-attachment : 78 
gill-spacing : 48 
gill-color : 63 
stem-root : 38 
stem-surface : 60 
stem-color : 59 
veil-type : 22 
veil-color : 24 
has-ring : 23 
ring-type : 40 
spore-print-color : 32 
habitat : 52 
season : 4 


# EDA

In [10]:
merge_data = pd.concat([train_data, test_data])
print(merge_data.shape)
merge_data.tail()

(5194909, 22)


,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
2077959,5194904,NaN,0.88,x,g,w,f,a,d,w,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,u
2077960,5194905,NaN,3.12,x,s,w,f,d,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
2077961,5194906,NaN,5.73,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
2077962,5194907,NaN,5.03,b,g,n,f,a,d,g,...,NaN,s,g,NaN,NaN,f,f,NaN,d,a
2077963,5194908,NaN,15.51,f,NaN,w,f,d,c,y,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,w


In [11]:
# id 삭제
merge_data.drop(labels="id", axis=1, inplace=True)
# NaN 값이 너무 많은 column 삭제
# nan_std = round(merge_data.shape[0] * 0.85)
# for col in merge_data.columns:
#     if merge_data[col].isnull().sum() > nan_std:
#         merge_data.drop(labels=col, axis=1, inplace=True)
#         categorical_col = categorical_col.drop(col)
#         print(f"{col} is Delete")

In [12]:
# NaN 값을 갖는 columns
((merge_data.isnull().sum() /merge_data.shape[0]) *100).sort_values()

stem-width               0.000000
season                   0.000000
stem-height              0.000019
cap-diameter             0.000212
does-bruise-or-bleed     0.000346
cap-color                0.000481
has-ring                 0.000828
stem-color               0.001136
habitat                  0.001347
cap-shape                0.001367
gill-color               0.002040
ring-type                4.140111
gill-attachment         16.819486
cap-surface             21.519665
class                   40.000008
gill-spacing            40.386270
stem-surface            63.568948
veil-color              87.914360
stem-root               88.452656
spore-print-color       91.422179
veil-type               94.882086
dtype: float64

In [13]:
# 범주형과 수치형 분류
categorical_col = train_data.select_dtypes(include=['object']).columns
numberical_col = train_data.select_dtypes(include=['float64']).columns

In [ ]:
# 각 column의 개별값들이 너무 많아서 시각화하기가 힘듬
# 따로따로 찾아야할 듯
categorical_col = train_data.select_dtypes(include=['object']).columns

print("unique value")

for col in categorical_col:
    print(f"{col} : {train_data[col].nunique()} ")

In [19]:
# target과 특성 간의 관련 정도를 시각화
# 
random_state = 42
target = train_data["class"]
mu_train_data = train_data[numberical_col]
mu_train_data = mu_train_data.fillna(0)

mu_info = mutual_info_classif(mu_train_data, target, random_state=random_state)
mu_df = pd.Series(mu_info)
mu_df.index = mu_train_data.columns
mu_info = pd.DataFrame(mu_df.sort_values(ascending=False), columns=["Numberic"])
mu_info.style.background_gradient("cool")

,Numberic
stem-width,0.068328
cap-diameter,0.042405
stem-height,0.041195


In [31]:
mu_train_data = train_data[categorical_col]
mu_train_data = mu_train_data.drop("class", axis=1)
mu_train_data = mu_train_data.fillna("NaN")

encoder = LabelEncoder()
for col in mu_train_data.columns:
    mu_train_data[col] = encoder.fit_transform(mu_train_data[col])

mu_train_data.head()
mu_info = mutual_info_classif(mu_train_data, target, random_state=random_state)
mu_df = pd.Series(mu_info)
mu_df.index = mu_train_data.columns
mu_info = pd.DataFrame(mu_df.sort_values(ascending=False), columns=["categorical"])
mu_info.style.background_gradient("cool")


,categorical
stem-root,0.237439
ring-type,0.235339
spore-print-color,0.233704
does-bruise-or-bleed,0.196800
has-ring,0.188190
stem-color,0.187821
habitat,0.181473
cap-shape,0.179206
cap-color,0.176721
stem-surface,0.159289


# Preprocessing

In [14]:
# class빼고 합치기 그리고 id 지우기
merge_data = pd.concat([train_data.drop("class", axis=1), test_data])
merge_data.drop("id", axis=1, inplace=True)
merge_data.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


### 전처리 파이프라인 설정

In [82]:
# 수치형 특성에 대하 전처리 파이프라인
numeric_transformer = Pipeline( steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

# 범주형 특성에 대한 전처리 파이프라인
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="NaN")),
    ("Label", LabelEncoder())
])

# 특성이 똑같아야 된다.
preprocessor = ColumnTransformer(
    [
        ("num", numeric_transformer, numberical_col),
        ("cat", categorical_transformer, ['cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color',
       'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season'])
    ]
)

In [81]:
categorical_col

Index(['class', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color',
       'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season'],
      dtype='object')

In [76]:
len(numberical_col)

3

# modeling

In [38]:
X = train_data.drop(["class", "id"], axis=1)
y = train_data["class"]

In [39]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [40]:
def mcc_metric(y_pred, dmatrix):
    y_true = dmatrix.get_label()
    y_pred = (y_pred > 0.5).astype(int) 
    mcc = matthews_corrcoef(y_true, y_pred)
    return 'mcc', mcc

## XGboost

In [78]:
X.columns

Index(['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [83]:
xgb_optuna_params = {
# 트리생성알고리즘
 'tree_method': 'gpu_hist' ,
    # 생성할 결정 트리의 수
 'n_estimators': 1696,
    # 가중치에 대한 L1 정규화 항
 'alpha': 4.956752183261538e-07,
    # 트리 학습할 때 사용할 데이터의 비율
 'subsample': 0.7349948172684168,
    #트리를 학습할 때 사용할 특성의 비율
 'colsample_bytree': 0.30171411525842506,
    # 최대 깊이 
 'max_depth': 15, 
    # 가중치의 최소 합계  
 'min_child_weight': 6,
    # 학습률
 'learning_rate': 0.013301072238797047,
    # 최소 손실 감소량
 'gamma': 5.634602153104516e-08
}

xgb_tuned = XGBClassifier(**xgb_optuna_params, random_state=random_state)


xgb_pipeline = make_pipeline(preprocessor, xgb_tuned)

# 교차 검증
accuracies = []

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X.iloc[train_idx].reset_index(drop=True), X.iloc[test_idx].reset_index(drop=True)
    y_train, y_test = y.iloc[train_idx].reset_index(drop=True), y.iloc[test_idx].reset_index(drop=True)


    # 학습
    xgb_pipeline.fit(X_train, y_train)
    
#     # 예측 평가
#     y_pred = xgb_pipeline.predict(X_test)
#     score = matthews_corrcoef(y_test, y_pred)
#     print('MCC:', score)
#     acc = accuracy_score(y_test, y_pred)
#     accuracies.append(acc)

# print("Accuracy scores for each fold: ", accuracies)
# print("Mean accuracy: ", sum(accuracies) / len(accuracies))

# cross_val_score(xgb_tuned, X, y, skf, scoring="accuracy")

TypeError: LabelEncoder.fit_transform() takes 2 positional arguments but 3 were given

## Catboost

## LGBM

## Ensemble